<h1>LangChain Beispiel</h1>

Wieso sollte man LangChain nutzen und nicht einfach ChatGPT? <br>
Oft will man ein eigenes LLM Model nutzen das mit den eigenen bestehenden Daten trainiert wurde. Das hat den Vorteil das keine sensitiven Daten nach außen dringen zu Webseiten, die ein Model bereitstellen und potenziell diese Daten die geschickt wurden speichert. Mit einem eigenen Model bleibt das Model Lokal auf dem PC oder im Unternehmensnetzwerk. Zudem haben Modelle die ChatGPT keinen direkten Zugriff auf unsere Daten.

Open-Source Modelle können auch einfach importiert und gespeichert werden, damit fällt das Haupttraining des Netzes weg (ggf. Fine-Tuning). Mit LangChain können solche Applikationen mit Modellen erstellt werden. Das Framework bietet verschiedenste Tools für den Umgang mit solchen Modellen.

Als Einstieg wollen wir ein Open-Source Modell nutzen und einfache Textabfragen abwickeln. Weitere Beispiele Folgen.

<i>Abb1</i>: ChatGPT nutzung. Kommunikation über Internet.

<img src="./data/img/1_lg.PNG" width=700 height=500>

*Um ChatGPT zu nutzen, muss ein OpenAI API Key bereitgestellt werden. Siehe OpenAI API <br>
*Kosten können anfallen.

Mehr über LangChain:

> https://python.langchain.com/v0.2/docs/introduction/ [Letzter Zugriff: 01.08.2024]

HuggingFace Modelle und Beispiele: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024] 

Es gibt eine große Anzahl von Modellen, die wir nutzen können. Als Einstieg nutzen wir das GPT2 Model.<br>
Auf der Webseite von HuggingFace wo die Modelle aufgelistet sind, befinden sich für jedes Model Beschreibungen und Beispiele.

Es sollte genug Speicher für ein Model bereitgestellt werden. 

GPT2: <br>
> https://huggingface.co/docs/transformers/model_doc/gpt2 [Letzter Zugriff: 01.08.2024]

Dieses Model wurde mit den Daten aus 8 Millionen Webseiten trainiert. <u>Das Ziel</u>: die nächsten Wörter vorhersagen.

Jedes Model kann andere Ziele haben.

In [19]:
# Imports.
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_core.output_parsers import StrOutputParser

In [20]:
# Erstelle Model.
model       = AutoModelForCausalLM.from_pretrained("gpt2")  # Model, welches wir nutzen wollen. 
tokenizer   = AutoTokenizer.from_pretrained("gpt2")  # Tokenizer des Models => Bereite Eingabe vor. 
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

Jedes Model ist anders und braucht einen anderen Input. Je nach Model und Aufgabe wird ein Tokenizer erstellt der den Text (oder andere Inputs) vorverarbeitet, um diese dann dem Model zu übergeben. Ohne ein Tokenizer müsste man alle Vorverarbeitungsschritte selber durchführen.

In [21]:
tokenizer("How are you doing?")

{'input_ids': [2437, 389, 345, 1804, 30], 'attention_mask': [1, 1, 1, 1, 1]}

In [22]:
prompt = "The weather is sunny today, so I will need "

# return_tensors: Rückgabetyp, hier: PyTorch. 
encoded_input  = tokenizer(prompt, return_tensors="pt")
encoded_input

{'input_ids': tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
input_ids      = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']

In [24]:
gen_tokens = model.generate(
    input_ids,  # Startpunkt für die Generierung.
    attention_mask=attention_mask,  # Ignoriere padding Tokens.  
    do_sample=True,   # Erlaubt Sampling:True: erlaubt kreative outputs. 
    temperature=0.5,  # Wie "kreativ" das Model sein soll => Randomness 
    max_length=25,    # Output Länge
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
gen_tokens

tensor([[  464,  6193,   318, 27737,  1909,    11,   523,   314,   481,   761,
           220,  3711,  8887,   290,   220,  3711,  6891,   284,   651,   832,
           428,    13,   314,   716,  1016]])

In [26]:
# batch_decode: Token IDs zurück zu Wörtern. 
# - Weitere Wörter werden generiert.
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)

The weather is sunny today, so I will need iced tea and iced coffee to get through this. I am going


Das war ein erstes einfaches Beispiel wie man ein Model nutzt. <br>

<h2>Promt Templates und Chains</h2>

Statt immer wieder den String zu ändern, können Templates verwendet werden, um Änderungen direkt im Text vorzunehmen.

In [27]:
from langchain_core.prompts.prompt import PromptTemplate

In [28]:
# Erstelle Template
my_template = PromptTemplate(
    input_variables= ['replace'],
    template=        "Today the weather is {replace}, so I will... "
)

# Teste Template
my_template.format(replace="sunny") 

'Today the weather is sunny, so I will... '

So können verschiedenen Elemente direkt im Text ersetzt werden. Wie der Name schon sagt, wird ein Template erstellt, wo bestimmte Lücken gefüllt werden können.

<i>Abb2</i>: Prompt Template, fülle die Lücken.

<img src="./data/img/2_lg.PNG" width=400 height=200>

In [29]:
# Weiteres Beispiel # 

template = "Tell me {x-number} of facts about the topic {topic}"  # Text Template.
promt_template = PromptTemplate.from_template(template)  # Erstelle Template.

promt =  promt_template.invoke({"x-number": 5, "topic": "dog" })
promt

StringPromptValue(text='Tell me 5 of facts about the topic dog')

Für den Anfang reicht es.

Dann gibt es noch Chains mit dem Operationen verkettet werden können. Das ist einer der Hauptbestandteile  von LangChain. Verschiedene Aufgaben können durch eine Verkettung zusammengefügt werden.

<i>Abb3</i>: Operationen Verketten.

<img src="./data/img/3_lg.PNG" width=650 height=450>

<i>Abb4</i>: Möglichkeiten im Überblick.

<img src="./data/img/4_lg.PNG" width=650 height=450>

Wie erwähnt gibt es viele Modelle die sich in der Größe und den Aufgaben unterscheiden. Hier nutzen wir das Model DistilGPT2

In [30]:
model_name = "distilgpt2"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForCausalLM.from_pretrained(model_name)

In [31]:
def make_prediction(text):
    # Parse Inputs.
    encoded_input  = tokenizer(text.to_string(), return_tensors="pt")
    input_ids      = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']
    # Model Config. 
    gen_tokens = model.generate(
        input_ids,  
        attention_mask=attention_mask,   
        do_sample=True,  
        temperature=0.7,  
        pad_token_id=tokenizer.eos_token_id,
        #max_length=50,   
        top_k=50,  
        top_p=0.95, 
        max_new_tokens=60,   
    )
    # Gebe Text zurück.
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    return gen_text

Je Umfangreicher das Model, desto bessere Antworten kann es liefern. Es gibt auch zahlreiche Parameter die eingestellt werden können.

In [32]:
# Template
template = "The weather is {weather}, what should I do today?"  # Text Template.
promt_template = PromptTemplate.from_template(template)  # Erstelle Template.
# Chain
chain = promt_template | make_prediction 
# Führe aus
result = chain.invoke({
    "weather":"sunny", 
})
print(result)

The weather is sunny, what should I do today?”


I was in my early 20s and was looking forward to my first day in a row with my wife and our 4-year-old daughter.
It was very cold and humid, and there was no food or the water. I was in the shower, so I


<h3>Erweitere Lineare Chain</h3>

Die Chain die wir haben `chain = promt_template | make_prediction ` kann z. B. durch Rannables erweitert werden.

In [222]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

# Erstelle Lambdas # 
upper_case  = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Count: { len(x.split()) }" )
# Eerstelle Chain
chain = promt_template | make_prediction | upper_case | count_words  # Erweitere Chain.

result = chain.invoke({
    "weather":"sunny", 
})
print(result)

KeyError: "Input to PromptTemplate is missing variables {'animal', 'like'}.  Expected: ['animal', 'like'] Received: ['weather']"

Durch Runnables kann so die Funktionalität erweitert werden.

<h3>Parallele Chains</h3>

<i>Abb5</i>: Aufbaubeispiel Parallele Chains.

<img src="./data/img/5_lg.PNG" width=400 height=600>

Man kann es sich so vorstellen, das man dem Model z. B. ChatGPT ein Produktname gibt und das Model soll getrennt Pros und Cons auflisten. Am Ende werden die Ergebnisse zu einer Liste vereinigt. Das erklärt ganz gut die Basisfunktionalität der parallelen Chains. 

Hier Nutzen wir das DialoGPT Model (small, medium, large).
> https://www.microsoft.com/en-us/research/project/large-scale-pretraining-for-response-generation/downloads/ [Letzter Zugriff: 07.08.2024]

Bei einem Chat-Model können drei Typen von Messages genutzt werden.<br>
In diesem Kontext gibt es 3 Arten von Texten.:<br>
- SystemMessage: Das den Kontext und das Verhalten beschreibt.
- Human Message: Was der Mensch sagt.
- AI Message: Was das Model ausgibt.

Für den Einstieg halten wir es simpel. Es ist keine Konversation wo wir die Nachrichten festhalten.

In [39]:
del model, tokenizer

In [92]:
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [199]:
def make_prediction(text):
   
    encoded_input  = tokenizer(text, return_tensors="pt")
    input_ids      = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']

    gen_tokens = model.generate(
        input_ids,
        attention_mask=attention_mask,
        do_sample=True,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
    )
    # Entferne Promt, damit nut Antwort sichtbar ist. 
    gen_text = tokenizer.decode(gen_tokens[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return gen_text
    

In [200]:
text = "Human: Let's talk about the weather, it is very sunny here, what should I do? \nAI:"

In [201]:
result = make_prediction(text)
print(result.strip())

I'm going to use a lot of sun, I will get better at snowboarding.


Für ein einfaches Beispiel werden zwei verschiedene Anfragen gestellt und später zusammengefügt.

In [211]:
# Erstelle Basistemplate # 
template = "Human: You are an Animal expert, is it true that {animal} like {like}? \nAI:" 
promt_template = PromptTemplate.from_template(template)  

result = make_prediction(promt_template.invoke({"animal": "dogs", "like": "milk"}).to_string())
print(result.strip())

No, humans are not animals. Humans can be animals.


In [215]:
def make_prediction_2(text):
   
    encoded_input  = tokenizer(text.to_string(), return_tensors="pt")
    input_ids      = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']

    gen_tokens = model.generate(
        input_ids,
        attention_mask=attention_mask,
        do_sample=True,
        temperature=0.75,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
    )
    # Entferne Promt, damit nut Antwort sichtbar ist. 
    gen_text = tokenizer.decode(gen_tokens[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return gen_text.strip()
    

Jetzt könnten wir diesen Text mit einem anderen größeren Model verarbeiten.<br>
- [Issue]: Gerade gibt es Probleme mit dem Anwenden größerer Modelle. Hier wird ein anderer Ansatz genutzt. <br>
  Statt ein Model zu nutzen werden Funktionen verwendet.

In [264]:
# Erstelle Funktionen
def count_words(text):
    return len(text.split())
def add_word_count(num):
    return num + 500

def text_transf_1(text:str):
    return text.upper()
def text_transf_2(text:str):
    return text.replace("a", '-')
    
# Erstelle Zweige.
# - Hier können wieder Chains erstellt werden. 
# - Weitere Templates oder Modelle. 
branch_1 = ( RunnableLambda(lambda x: count_words(x)   ))  | add_word_count
branch_2 = ( RunnableLambda(lambda x: text_transf_1(x)  )) | text_transf_2 

In [265]:
chain = (
    promt_template | make_prediction_2  # Oder auch andere diverse Vorbereitungen.
    # - Parallel:
    | RunnableParallel(branches={"branch_1": branch_1, "branch_2": branch_2 })  # Gebe Branches an.
    # Weitere Schritte...
    | RunnableLambda(lambda x: f"Result Branch_1: {x['branches']['branch_1']}\n" +
                    f"Result Branch_2: {x['branches']['branch_2']}")
    # Mit Keys zugreifen. 
)

In [266]:
print(chain.invoke({"animal": "dogs", "like": "milk"}))

Result Branch_1: 514
Result Branch_2: NO, THAT'S NOT TRUE. HUMANS : MILK IS THE BEST FOOD IN THE WORLD.


Durch diese parallelen Chains können beliebige viele Operationen angehangen werden die nach n-Branches zusammengefügt werden, um ein Ergebnis zu liefern.

<h2>Branching</h2>

Hier geht es darum basierend auf einem Argument einen Branch auszuwählen. Man kann es sich wie ein If-Statement vorstellen.

<i>Abb6</i>: Branching.

<img src="./data/img/6_lg.PNG" width=450 height=650>